# ***0. Import libraries***

In [1]:
import re
import gensim
from gensim.utils import simple_preprocess
import jieba
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## ***1. Remove spaces between text***

In [11]:
df = pd.read_csv('王者荣耀女玩家_all_comments.csv', encoding="utf-8-sig")
df['newcontent'] = df['content'] 
df

,note_id,is_root,is_author,comment_id,comment_time,ip_location,content,like_cnt,at_list,hashtag_list,sub_comment_cnt,target_comment_id,target_comment_user_name,target_comment_user_id,comment_user_name,comment_user_id,newcontent
0,5d22801f000000002603488b,0.0,1.0,5d22e21b0000000027025006,2019-07-08 14:26:35,NaN,😂,0,[],"[\n ""is_author""\n]",-1,5d22c4800000000028033d15,scream🐣,58cce41d82ec391358e047d4,一盏鲸鱼灯,5d0b499a000000001601c8cc,😂
1,5d22801f000000002603488b,0.0,1.0,5d22e1e90000000026026489,2019-07-08 14:25:45,NaN,…嗯…,0,[],"[\n ""is_author""\n]",-1,5d22c25f000000002701154c,Syy,556f999d88a0915a67ea06fd,一盏鲸鱼灯,5d0b499a000000001601c8cc,…嗯…
2,5d22801f000000002603488b,0.0,0.0,5d22c4800000000028033d15,2019-07-08 12:20:16,NaN,可是我很争气，多数都是第四名哈哈哈哈哈,0,[],[],-1,5d22af3e0000000026007b85,一盏鲸鱼灯,5d0b499a000000001601c8cc,scream🐣,58cce41d82ec391358e047d4,可是我很争气，多数都是第四名哈哈哈哈哈
3,5d22801f000000002603488b,1.0,0.0,5d22c25f000000002701154c,2019-07-08 12:11:11,NaN,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。,0,[],[],1,NaN,NaN,NaN,Syy,556f999d88a0915a67ea06fd,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。
4,5d22801f000000002603488b,0.0,1.0,5d22bb09000000002802e6f8,2019-07-08 11:39:53,NaN,哈哈哈[偷笑R],0,[],"[\n ""is_author""\n]",-1,5d22b0190000000028027f23,这不是去幼儿园的车,5d0f40de0000000010032912,一盏鲸鱼灯,5d0b499a000000001601c8cc,哈哈哈[偷笑R]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415972,65505bda000000003300649d,0.0,0.0,65505eef000000001b03d376,2023-11-12 13:13:19,江苏,百度一下，百度教程很详细,0,[],[],-1,65505ea6000000000f028aa9,K^,613028a600000000020253e3,money come,5d2993a000000000110219cc,百度一下，百度教程很详细
415973,65505bda000000003300649d,0.0,0.0,65505ea6000000000f028aa9,2023-11-12 13:12:06,河南,怎么改性别,0,[],[],-1,65505dda000000001b03f9f3,money come,5d2993a000000000110219cc,K^,613028a600000000020253e3,怎么改性别
415974,65505bda000000003300649d,1.0,0.0,65505dda000000001b03f9f3,2023-11-12 13:08:42,江苏,把自己改成男的两个月了，这次刷了安琪拉[大笑R],0,[],[],2,NaN,NaN,NaN,money come,5d2993a000000000110219cc,把自己改成男的两个月了，这次刷了安琪拉[大笑R]
415975,65505bda000000003300649d,1.0,0.0,65505d47000000003300696f,2023-11-12 13:06:15,广东,我想知道一千之后还能刷出皮肤了[生气R]我一直都刷不出来,0,[],[],0,NaN,NaN,NaN,郝凶,5eaa56800000000001003685,我想知道一千之后还能刷出皮肤了[生气R]我一直都刷不出来


In [12]:
def sent_to_words_space(sentences):
    for sent in sentences:
        sent = sent.lower()
        sent = re.sub("[~^0-9]", "", sent)
        sent = re.sub(r'http\S+', '', sent)
        sent = re.sub(r'https\S+', '', sent)
        sent = re.sub('<[^>]+>', '', sent)
        sent = re.sub('<[^<]+?>', '', sent)
        sent = re.sub('\S*@\S*\s?', '', sent)
        sent = re.sub('\s+', ' ', sent)
        sent = re.sub("\'", "", sent)
        sent = re.sub(r'[^\w\s]', ' ', sent)

        yield sent

In [14]:
df['content'] = df['content'].astype(str)

msg_space = df.content.values.tolist()

msg_space_words = list(sent_to_words_space(msg_space))

print(msg_space_words[:1])

[' ']


In [15]:
df['newcontent'] = msg_space_words
df.to_csv("femalecomment_only.csv", index=False, encoding="utf-8")
df.head(5)

,note_id,is_root,is_author,comment_id,comment_time,ip_location,content,like_cnt,at_list,hashtag_list,sub_comment_cnt,target_comment_id,target_comment_user_name,target_comment_user_id,comment_user_name,comment_user_id,newcontent
0,5d22801f000000002603488b,0.0,1.0,5d22e21b0000000027025006,2019-07-08 14:26:35,NaN,😂,0,[],"[\n ""is_author""\n]",-1,5d22c4800000000028033d15,scream🐣,58cce41d82ec391358e047d4,一盏鲸鱼灯,5d0b499a000000001601c8cc,
1,5d22801f000000002603488b,0.0,1.0,5d22e1e90000000026026489,2019-07-08 14:25:45,NaN,…嗯…,0,[],"[\n ""is_author""\n]",-1,5d22c25f000000002701154c,Syy,556f999d88a0915a67ea06fd,一盏鲸鱼灯,5d0b499a000000001601c8cc,嗯
2,5d22801f000000002603488b,0.0,0.0,5d22c4800000000028033d15,2019-07-08 12:20:16,NaN,可是我很争气，多数都是第四名哈哈哈哈哈,0,[],[],-1,5d22af3e0000000026007b85,一盏鲸鱼灯,5d0b499a000000001601c8cc,scream🐣,58cce41d82ec391358e047d4,可是我很争气 多数都是第四名哈哈哈哈哈
3,5d22801f000000002603488b,1.0,0.0,5d22c25f000000002701154c,2019-07-08 12:11:11,NaN,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。,0,[],[],1,NaN,NaN,NaN,Syy,556f999d88a0915a67ea06fd,知道自己菜 就别拿c位咯 不是每次都能这么幸运的
4,5d22801f000000002603488b,0.0,1.0,5d22bb09000000002802e6f8,2019-07-08 11:39:53,NaN,哈哈哈[偷笑R],0,[],"[\n ""is_author""\n]",-1,5d22b0190000000028027f23,这不是去幼儿园的车,5d0f40de0000000010032912,一盏鲸鱼灯,5d0b499a000000001601c8cc,哈哈哈 偷笑r


## ***2. Preliminary dictionary-based tokenization***

In [16]:
df = pd.read_csv("femalecomment_only.csv", encoding='utf-8-sig')
df.head(1)

/var/folders/g3/whtrsvys0vg0n49n23hq5v180000gp/T/ipykernel_21929/2571254078.py:1: DtypeWarning: Columns (2,5,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("femalecomment_only.csv", encoding='utf-8-sig')


,note_id,is_root,is_author,comment_id,comment_time,ip_location,content,like_cnt,at_list,hashtag_list,sub_comment_cnt,target_comment_id,target_comment_user_name,target_comment_user_id,comment_user_name,comment_user_id,newcontent
0,5d22801f000000002603488b,0.0,1.0,5d22e21b0000000027025006,2019-07-08 14:26:35,NaN,😂,0.0,[],"[\n ""is_author""\n]",-1,5d22c4800000000028033d15,scream🐣,58cce41d82ec391358e047d4,一盏鲸鱼灯,5d0b499a000000001601c8cc,


In [17]:
df['newcontent'] = df['content'].astype(str)
data = df['newcontent'].values.tolist()
print("Total Number of Documents:", len(data))

Total Number of Documents: 576029


In [18]:
jieba.load_userdict("dict.txt")


def stopwordslist():
    stopwords = [line.strip() for line in open('new_stopwords.txt', 'r', encoding='utf-8').readlines()]
    return stopwords


# (Optional) 只要中文
def is_chinese(uchar):
    if uchar >= '\u4e00' and uchar <= '\u9fa5':
        return True
    else:
        return False


# 定義停詞函數 對句子進行中文分詞
def seg_depart(sentence):
    # 對文檔中的每一行進行中文分詞
    sentence_depart = jieba.cut(sentence.strip())
    # 輸出結果為outstr
    stopwords = stopwordslist()
    outstr = ''
    # 去停用詞
    for word in sentence_depart:
        if word not in stopwords and (len(word.strip()) > 0) and is_chinese(word):
            outstr += word
            outstr += " "
    return outstr


# 分詞後的結果
result_fenci = []
for i in data:
    result_fenci.append([i, seg_depart(i)])

In [19]:
result_fenci = [i[1].split(' ')[:-1] for i in result_fenci]
print(result_fenci[:5])

[[], [], ['争气', '多数', '第四名'], ['知道', '菜', '就别', '位咯', '每次', '都能', '幸运'], ['偷笑']]


## ***3. Implement bigram and trigram to tokens***

In [20]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(result_fenci, min_count=10, threshold=10)  # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[result_fenci], min_count=10, threshold=10)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


def process_words(texts):
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]

    return texts


ngram_fenci = process_words(result_fenci)  # processed Text Data!

print(ngram_fenci[:10])

[[], [], ['争气', '多数', '第四名'], ['知道', '菜', '就别', '位咯', '每次', '都能', '幸运'], ['偷笑'], ['吖'], [], ['有空', '再来', '叭', '复习'], ['玩_笑_哭'], ['笑_哭']]


In [21]:
df['tokenz'] = ngram_fenci
df

,note_id,is_root,is_author,comment_id,comment_time,ip_location,content,like_cnt,at_list,hashtag_list,sub_comment_cnt,target_comment_id,target_comment_user_name,target_comment_user_id,comment_user_name,comment_user_id,newcontent,tokenz
0,5d22801f000000002603488b,0.0,1.0,5d22e21b0000000027025006,2019-07-08 14:26:35,NaN,😂,0.0,[],"[\n ""is_author""\n]",-1,5d22c4800000000028033d15,scream🐣,58cce41d82ec391358e047d4,一盏鲸鱼灯,5d0b499a000000001601c8cc,😂,[]
1,5d22801f000000002603488b,0.0,1.0,5d22e1e90000000026026489,2019-07-08 14:25:45,NaN,…嗯…,0.0,[],"[\n ""is_author""\n]",-1,5d22c25f000000002701154c,Syy,556f999d88a0915a67ea06fd,一盏鲸鱼灯,5d0b499a000000001601c8cc,…嗯…,[]
2,5d22801f000000002603488b,0.0,0.0,5d22c4800000000028033d15,2019-07-08 12:20:16,NaN,可是我很争气，多数都是第四名哈哈哈哈哈,0.0,[],[],-1,5d22af3e0000000026007b85,一盏鲸鱼灯,5d0b499a000000001601c8cc,scream🐣,58cce41d82ec391358e047d4,可是我很争气，多数都是第四名哈哈哈哈哈,"[争气, 多数, 第四名]"
3,5d22801f000000002603488b,1.0,0.0,5d22c25f000000002701154c,2019-07-08 12:11:11,NaN,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。,0.0,[],[],1,NaN,NaN,NaN,Syy,556f999d88a0915a67ea06fd,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。,"[知道, 菜, 就别, 位咯, 每次, 都能, 幸运]"
4,5d22801f000000002603488b,0.0,1.0,5d22bb09000000002802e6f8,2019-07-08 11:39:53,NaN,哈哈哈[偷笑R],0.0,[],"[\n ""is_author""\n]",-1,5d22b0190000000028027f23,这不是去幼儿园的车,5d0f40de0000000010032912,一盏鲸鱼灯,5d0b499a000000001601c8cc,哈哈哈[偷笑R],[偷笑]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576024,65505bda000000003300649d,0.0,0.0,65505eef000000001b03d376,2023-11-12 13:13:19,江苏,百度一下，百度教程很详细,0,[],[],-1,65505ea6000000000f028aa9,K^,613028a600000000020253e3,money come,5d2993a000000000110219cc,百度一下，百度教程很详细,"[百度, 百度, 教程, 详细]"
576025,65505bda000000003300649d,0.0,0.0,65505ea6000000000f028aa9,2023-11-12 13:12:06,河南,怎么改性别,0,[],[],-1,65505dda000000001b03f9f3,money come,5d2993a000000000110219cc,K^,613028a600000000020253e3,怎么改性别,"[改, 性别]"
576026,65505bda000000003300649d,1.0,0.0,65505dda000000001b03f9f3,2023-11-12 13:08:42,江苏,把自己改成男的两个月了，这次刷了安琪拉[大笑R],0,[],[],2,NaN,NaN,NaN,money come,5d2993a000000000110219cc,把自己改成男的两个月了，这次刷了安琪拉[大笑R],"[改成, 男, 两个月, 刷了, 安琪拉, 大笑]"
576027,65505bda000000003300649d,1.0,0.0,65505d47000000003300696f,2023-11-12 13:06:15,广东,我想知道一千之后还能刷出皮肤了[生气R]我一直都刷不出来,0,[],[],0,NaN,NaN,NaN,郝凶,5eaa56800000000001003685,我想知道一千之后还能刷出皮肤了[生气R]我一直都刷不出来,"[想知道, 一千, 还能, 刷出, 皮肤, 生气, 刷, 不出来]"


## ***4. Removal of unnecessary symbols***

In [22]:
def sent_to_words_space(sentences):
    for sent in sentences:
        sent = re.sub(r'[^\w\s]', '', sent)  # remove punctuations
        sent = sent.replace('nan', '')
        sent = sent.replace('_', '')
        yield (sent)


df['tokenz'] = df['tokenz'].astype(str)

msg_space = df.tokenz.values.tolist()
msg_space_words = list(sent_to_words_space(msg_space))

print(msg_space_words[:1])

df['tokenz'] = msg_space_words
df

['']


,note_id,is_root,is_author,comment_id,comment_time,ip_location,content,like_cnt,at_list,hashtag_list,sub_comment_cnt,target_comment_id,target_comment_user_name,target_comment_user_id,comment_user_name,comment_user_id,newcontent,tokenz
0,5d22801f000000002603488b,0.0,1.0,5d22e21b0000000027025006,2019-07-08 14:26:35,NaN,😂,0.0,[],"[\n ""is_author""\n]",-1,5d22c4800000000028033d15,scream🐣,58cce41d82ec391358e047d4,一盏鲸鱼灯,5d0b499a000000001601c8cc,😂,
1,5d22801f000000002603488b,0.0,1.0,5d22e1e90000000026026489,2019-07-08 14:25:45,NaN,…嗯…,0.0,[],"[\n ""is_author""\n]",-1,5d22c25f000000002701154c,Syy,556f999d88a0915a67ea06fd,一盏鲸鱼灯,5d0b499a000000001601c8cc,…嗯…,
2,5d22801f000000002603488b,0.0,0.0,5d22c4800000000028033d15,2019-07-08 12:20:16,NaN,可是我很争气，多数都是第四名哈哈哈哈哈,0.0,[],[],-1,5d22af3e0000000026007b85,一盏鲸鱼灯,5d0b499a000000001601c8cc,scream🐣,58cce41d82ec391358e047d4,可是我很争气，多数都是第四名哈哈哈哈哈,争气 多数 第四名
3,5d22801f000000002603488b,1.0,0.0,5d22c25f000000002701154c,2019-07-08 12:11:11,NaN,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。,0.0,[],[],1,NaN,NaN,NaN,Syy,556f999d88a0915a67ea06fd,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。,知道 菜 就别 位咯 每次 都能 幸运
4,5d22801f000000002603488b,0.0,1.0,5d22bb09000000002802e6f8,2019-07-08 11:39:53,NaN,哈哈哈[偷笑R],0.0,[],"[\n ""is_author""\n]",-1,5d22b0190000000028027f23,这不是去幼儿园的车,5d0f40de0000000010032912,一盏鲸鱼灯,5d0b499a000000001601c8cc,哈哈哈[偷笑R],偷笑
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576024,65505bda000000003300649d,0.0,0.0,65505eef000000001b03d376,2023-11-12 13:13:19,江苏,百度一下，百度教程很详细,0,[],[],-1,65505ea6000000000f028aa9,K^,613028a600000000020253e3,money come,5d2993a000000000110219cc,百度一下，百度教程很详细,百度 百度 教程 详细
576025,65505bda000000003300649d,0.0,0.0,65505ea6000000000f028aa9,2023-11-12 13:12:06,河南,怎么改性别,0,[],[],-1,65505dda000000001b03f9f3,money come,5d2993a000000000110219cc,K^,613028a600000000020253e3,怎么改性别,改 性别
576026,65505bda000000003300649d,1.0,0.0,65505dda000000001b03f9f3,2023-11-12 13:08:42,江苏,把自己改成男的两个月了，这次刷了安琪拉[大笑R],0,[],[],2,NaN,NaN,NaN,money come,5d2993a000000000110219cc,把自己改成男的两个月了，这次刷了安琪拉[大笑R],改成 男 两个月 刷了 安琪拉 大笑
576027,65505bda000000003300649d,1.0,0.0,65505d47000000003300696f,2023-11-12 13:06:15,广东,我想知道一千之后还能刷出皮肤了[生气R]我一直都刷不出来,0,[],[],0,NaN,NaN,NaN,郝凶,5eaa56800000000001003685,我想知道一千之后还能刷出皮肤了[生气R]我一直都刷不出来,想知道 一千 还能 刷出 皮肤 生气 刷 不出来


## ***5. Word count and stop word filtering for tokenz column***

In [23]:
df['tokenz'] = df['tokenz'].astype(str)
data = df['tokenz'].values.tolist()
print("Total Number of Documents:", len(data))

Total Number of Documents: 576029


In [24]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = gensim.utils.simple_preprocess(str(sent), min_len=2, deacc=True)
        yield sent

data_words = list(sent_to_words(data))
print(data_words[:5])

[[], [], ['争气', '多数', '第四名'], ['知道', '就别', '位咯', '每次', '都能', '幸运'], ['偷笑']]


In [25]:
stop_words = [line.strip() for line in open('cn_stopwords.txt', 'r', encoding='UTF-8').readlines()]

def process_words(texts, stop_words=stop_words):
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    return texts

data_ready = process_words(data_words)
print(data_ready[:5])

df['tokenz'] = data_ready

[[], [], ['争气', '多数', '第四名'], ['知道', '就别', '位咯', '每次', '都能', '幸运'], ['偷笑']]


In [26]:
def sent_to_words_space(sentences):
    for sent in sentences:
        sent = re.sub(r'[^\w\s]', '', sent)  # remove punctuations
        yield sent

df['tokenz'] = df['tokenz'].astype(str)

msg_space = df.tokenz.values.tolist()
msg_space_words = list(sent_to_words_space(msg_space))

print(msg_space_words[:1])

df['tokenz'] = msg_space_words
df

['']


,note_id,is_root,is_author,comment_id,comment_time,ip_location,content,like_cnt,at_list,hashtag_list,sub_comment_cnt,target_comment_id,target_comment_user_name,target_comment_user_id,comment_user_name,comment_user_id,newcontent,tokenz
0,5d22801f000000002603488b,0.0,1.0,5d22e21b0000000027025006,2019-07-08 14:26:35,NaN,😂,0.0,[],"[\n ""is_author""\n]",-1,5d22c4800000000028033d15,scream🐣,58cce41d82ec391358e047d4,一盏鲸鱼灯,5d0b499a000000001601c8cc,😂,
1,5d22801f000000002603488b,0.0,1.0,5d22e1e90000000026026489,2019-07-08 14:25:45,NaN,…嗯…,0.0,[],"[\n ""is_author""\n]",-1,5d22c25f000000002701154c,Syy,556f999d88a0915a67ea06fd,一盏鲸鱼灯,5d0b499a000000001601c8cc,…嗯…,
2,5d22801f000000002603488b,0.0,0.0,5d22c4800000000028033d15,2019-07-08 12:20:16,NaN,可是我很争气，多数都是第四名哈哈哈哈哈,0.0,[],[],-1,5d22af3e0000000026007b85,一盏鲸鱼灯,5d0b499a000000001601c8cc,scream🐣,58cce41d82ec391358e047d4,可是我很争气，多数都是第四名哈哈哈哈哈,争气 多数 第四名
3,5d22801f000000002603488b,1.0,0.0,5d22c25f000000002701154c,2019-07-08 12:11:11,NaN,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。,0.0,[],[],1,NaN,NaN,NaN,Syy,556f999d88a0915a67ea06fd,知道自己菜 就别拿c位咯 不是每次都能这么幸运的。,知道 就别 位咯 每次 都能 幸运
4,5d22801f000000002603488b,0.0,1.0,5d22bb09000000002802e6f8,2019-07-08 11:39:53,NaN,哈哈哈[偷笑R],0.0,[],"[\n ""is_author""\n]",-1,5d22b0190000000028027f23,这不是去幼儿园的车,5d0f40de0000000010032912,一盏鲸鱼灯,5d0b499a000000001601c8cc,哈哈哈[偷笑R],偷笑
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576024,65505bda000000003300649d,0.0,0.0,65505eef000000001b03d376,2023-11-12 13:13:19,江苏,百度一下，百度教程很详细,0,[],[],-1,65505ea6000000000f028aa9,K^,613028a600000000020253e3,money come,5d2993a000000000110219cc,百度一下，百度教程很详细,百度 百度 教程 详细
576025,65505bda000000003300649d,0.0,0.0,65505ea6000000000f028aa9,2023-11-12 13:12:06,河南,怎么改性别,0,[],[],-1,65505dda000000001b03f9f3,money come,5d2993a000000000110219cc,K^,613028a600000000020253e3,怎么改性别,性别
576026,65505bda000000003300649d,1.0,0.0,65505dda000000001b03f9f3,2023-11-12 13:08:42,江苏,把自己改成男的两个月了，这次刷了安琪拉[大笑R],0,[],[],2,NaN,NaN,NaN,money come,5d2993a000000000110219cc,把自己改成男的两个月了，这次刷了安琪拉[大笑R],改成 两个月 刷了 安琪拉 大笑
576027,65505bda000000003300649d,1.0,0.0,65505d47000000003300696f,2023-11-12 13:06:15,广东,我想知道一千之后还能刷出皮肤了[生气R]我一直都刷不出来,0,[],[],0,NaN,NaN,NaN,郝凶,5eaa56800000000001003685,我想知道一千之后还能刷出皮肤了[生气R]我一直都刷不出来,想知道 一千 还能 刷出 皮肤 生气 不出来


In [27]:
df.to_csv("commentstoken.csv", encoding='utf-8', index=False)

In [1]:
import csv

with open('T_10.txt', 'r') as input_file, open('T_10.csv', 'w', newline='') as output_file:
    writer = csv.writer(output_file)

    writer.writerow(['Topic#', 'Highest Prob', 'FREX', 'Lift', 'Score'])

    topic_number = ""
    highest_prob = ""
    frex = ""
    lift = ""
    score = ""

    for line in input_file:
        line = line.strip()

        if line.startswith('Topic'):
            # If we have data for the previous topic, write it to the output file
            if topic_number != "":
                writer.writerow([topic_number, highest_prob, frex, lift, score])

            # Extract the topic number and highest probability for the new topic
            parts = line.split()
            topic_number = parts[1]
            highest_prob = ""
            frex = ""
            lift = ""
            score = ""

        elif line.startswith('Highest Prob'):
            # Extract the highest probability value
            parts = line.split(':')
            highest_prob = parts[1].strip()

        elif line.startswith('FREX'):
            # Extract the FREX value
            parts = line.split(':')
            frex = parts[1].strip()

        elif line.startswith('Lift'):
            # Extract the Lift value
            parts = line.split(':')
            lift = parts[1].strip()

        elif line.startswith('Score'):
            # Extract the Score value
            parts = line.split(':')
            score = parts[1].strip()

    # Write the data for the last topic to the output file
    writer.writerow([topic_number, highest_prob, frex, lift, score])
